In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
data=[]
pages = range(0,10)
URL = "https://www.ambitionbox.com/list-of-companies?page=1"
driver.get(URL)
next = False
nom = 1
# wait for company cards
for page in pages:
    if next:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.page-nav-btn.router-link-active"))
        )
        next_button.click()

    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.companyCardWrapper"))
    )
    # time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    for card in soup.select("div.companyCardWrapper"):
    # Company short name
        name_tag = card.select_one("h2.companyCardWrapper__companyName")
        name = name_tag.get_text(strip=True) if name_tag else None
        
        # Full name
        full_name = card.find("meta", {"itemprop": "alternateName"})
        full_name = full_name["content"] if full_name else None
        
        # Rating
        rating_tag = card.select_one("div.rating_text")
        rating = rating_tag.get_text(strip=True) if rating_tag else None
        
        # Reviews (ActionWrapper with title 'Reviews')
        reviews_tag = card.find("span", string="Reviews")
        reviews = reviews_tag.find_previous("span").get_text(strip=True) if reviews_tag else None
        
        # Jobs
        jobs_tag = card.find("span", string="Jobs")
        jobs = jobs_tag.find_previous("span").get_text(strip=True) if jobs_tag else None
        
        data.append({
            "Company": name,
            "Full Name": full_name,
            "Rating": rating,
            "Reviews": reviews,
            "Jobs": jobs
        })
    print(nom)
    nom = nom + 1
    next = True
df = pd.DataFrame(data)


driver.quit()


1
2
3
4
5
6
7
8
9
10


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Company    200 non-null    object
 1   Full Name  200 non-null    object
 2   Rating     200 non-null    object
 3   Reviews    200 non-null    object
 4   Jobs       200 non-null    object
dtypes: object(5)
memory usage: 7.9+ KB
